# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [26]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BenchmarkTools

Threads.nthreads() == 1 && 
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`");
println("Number of threads: $(Threads.nthreads())")

  Activating 

Number of threads: 4


project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [27]:
function LoopFields(c, model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(c, model, Δmax) = Spectrum(LoopFields(c, model), Δmax, interchiral=true);

function solve(;indices, signature, parity, even_spin=(), benchmark=false)
    ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, :On, Δmax)
    Seven = Spectrum([V for V in SOn.fields if spin(V) % 2 == 0], Δmax, interchiral=true)

    specs = (; (chan =>
        if chan in even_spin
            Seven
        else
            SOn
        end
                for chan in (:s, :t, :u)
    )...)

    chan_spectra = ChannelSpectra(
        co, specs, signature;
        interchiral=true, Δmax=Δmax, parity=1
    )

    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
    if benchmark
        println("precomputation of blocks coefficients")
        @btime ChannelSpectra(
            $co, $SOn, $signature;
            interchiral=true, Δmax=Δmax, parity=1,
            exclude=(u=[Field(c, r=2, s=0)],)
        )
        println("time to compute all positions' cache")
        @btime BootstrapSystem($chan_spectra)
        println("time to evaluate blocks")
        @btime evaluate_blocks!($sys)
        println("time for the linear solver")
        @btime solve!($sys)
    end
    return sys
end

solve (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [ ]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.1" * im))
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
sig = (s=0, t=1, u=1)
sol = solve(indices=ind, signature=sig, parity=1, even_spin=(:s,), benchmark=true)
println("total time to setup and solve the crossing equations:")
@btime solve(indices=ind, signature=sig, parity=1, even_spin=(:s,))
println(sol.consts)



precomputation of blocks coefficients
  675.358 ms (18403246 allocations: 833.51 MiB)
time to compute all positions' cache
  151.425 ms (6207298 allocations: 260.33 MiB)
time to evaluate blocks
  1.229 s (22168554 allocations: 949.47 MiB)
time for the linear solver
  616.565 ms (11998371 allocations: 495.38 MiB)
total time to setup and solve the crossing equations:
  3.005 s (60432304 allocations: 2.56 GiB)
70×4 DataFrame
 Row │ Channel  Field          StructureConstant           RelativeError 
     │ Symbol   String         Complex                     Float32       
─────┼───────────────────────────────────────────────────────────────────
   1 │ s        V_{<1, 1>}              1.0+0.0im            0.0
   2 │ s        V_{(1, 0)}        -0.182995+0.243693im       6.39574e-20
   3 │ s        V_{(2, 0)}     -0.000165018+0.000981007im    2.40846e-17
   4 │ s        V_{(2, 1)}      0.000133433-0.000458461im    5.99279e-18
   5 │ s        V_{(3, 0)}      2.60868e-08+1.31146e-08im    1.60147

Benchmarks:
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [19]:
ind = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
sig = (s=0, t=3//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1, even_spin=(:s,)).consts)

56×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        <1, 1>                        1.0+0.0im            0.0
   2 │ s        V_{(1, 0)}              -0.256792+0.389886im       1.54121e-19
   3 │ s        V_{(2, 0)}             0.00281023-0.00250618im     1.4445e-17
   4 │ s        V_{(2, 1)}            7.88314e-22+2.00731e-20im    7.605
   5 │ s        V_{(3, 0)}           -1.19812e-06-9.08135e-07im    2.15004e-13
   6 │ s        V_{(3, 2//3)}         8.21729e-07+4.50803e-07im    1.86141e-14
   7 │ s        V_{(4, 0)}           -1.17948e-12+4.35239e-13im    3.36782e-6
   8 │ s        V_{(4, 1//2)}         6.59515e-13-2.50726e-13im    1.52767e-6
   9 │ s        V_{(4, 1)}           -6.96013e-19+4.59967e-19im    1.21672
  10 │ s        V_{(5, 0)}           -2.08004e-17-3.461

In [22]:
sig = (s=1, t=1//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1).consts)

68×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1, 0)}                    1.0+0.0im            0.0
   2 │ s        V_{(1, 1)}               0.502183-0.0830317im      1.14837e-19
   3 │ s        V_{(2, 0)}             0.00239155+0.0255139im      1.00888e-17
   4 │ s        V_{(2, 1//2)}          0.00273704+0.0200265im      1.2884e-17
   5 │ s        V_{(2, 1)}            3.81676e-20-3.64733e-20im    0.501164
   6 │ s        V_{(3, 0)}           -6.40312e-07-1.92164e-06im    3.34065e-12
   7 │ s        V_{(3, 1//3)}        -7.07647e-07-9.43683e-07im    4.17982e-12
   8 │ s        V_{(3, 2//3)}        -6.00257e-07+2.35486e-07im    1.94844e-12
   9 │ s        V_{(3, 1)}             4.3925e-19+8.36611e-19im    0.429026
  10 │ s        V_{(4, 0)}            1.40161e-12

In [23]:
sig = (s=1, t=3//2, u=1//2)
println(solve(indices=ind, signature=sig, parity=1).consts)

68×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1, 0)}                    1.0+0.0im            0.0
   2 │ s        V_{(1, 1)}              -0.502183+0.0830317im      7.95465e-19
   3 │ s        V_{(2, 0)}             0.00239155+0.0255139im      5.95221e-17
   4 │ s        V_{(2, 1//2)}         -0.00273704-0.0200265im      1.0174e-17
   5 │ s        V_{(2, 1)}            1.80396e-19-4.82125e-19im    0.132952
   6 │ s        V_{(3, 0)}           -6.40312e-07-1.92164e-06im    7.5041e-12
   7 │ s        V_{(3, 1//3)}         7.07647e-07+9.43683e-07im    8.19263e-12
   8 │ s        V_{(3, 2//3)}        -6.00257e-07+2.35486e-07im    6.4147e-12
   9 │ s        V_{(3, 1)}            6.13774e-18+3.36869e-18im    0.076444
  10 │ s        V_{(4, 0)}            1.44379e-12+1

# Torus one point functions

In [24]:
V1 = Field(c, r=1, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1//2, t=0, u=1//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
println(sys.consts)

70×4 DataFrame
 Row │ Channel  Field                StructureConstant           RelativeError 
     │ Symbol   String               Complex                     Float32       
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ s        V_{(1//2, 0)}                 1.0+0.0im               0.0
   2 │ s        V_{(3//2, 0)}             4.67783+3.19459im           0.20694
   3 │ s        V_{(3//2, 2//3)}        -0.260871+0.111512im          0.938826
   4 │ s        V_{(5//2, 0)}             2388.32-2737.44im           0.845179
   5 │ s        V_{(5//2, 2//5)}         -1936.95+2272.92im           0.809203
   6 │ s        V_{(5//2, 4//5)}          691.876-835.488im           0.873842
   7 │ s        V_{(7//2, 0)}         -3.5442e+07-1.83382e+08im       0.541238
   8 │ s        V_{(7//2, 2//7)}       3.1962e+07+1.65052e+08im       0.541235
   9 │ s        V_{(7//2, 4//7)}     -2.21867e+07-1.13883e+08im       0.541267
  10 │ s        V_{(7//2, 6//7)}      7.